# Quiz Access Management Tool

This notebook provides utilities to easily manage access control for quizzes in the quiz_engine_pro module.
It focuses on making it simple to grant and revoke quiz access for Portal users.

## Setup Connection to Odoo

First, we need to establish a connection to your Odoo instance.

In [ ]:
import xmlrpc.client
import pandas as pd
from IPython.display import display, HTML

# Configure Odoo connection
URL = "https://your-odoo-instance.com"
DB = "your_database"
USERNAME = "admin"  # Use an admin user for access control management
PASSWORD = "your_password"  # Replace with your actual password

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{URL}/xmlrpc/2/common')
uid = common.authenticate(DB, USERNAME, PASSWORD, {})
models = xmlrpc.client.ServerProxy(f'{URL}/xmlrpc/2/object')

if uid:
    print(f"Successfully connected to {URL} as {USERNAME} (uid: {uid})")
else:
    print("Failed to authenticate. Please check your credentials.")

## List Available Quizzes

Let's start by listing all quizzes and their current access modes.

In [ ]:
def get_quizzes():
    """Get all quizzes with their access information"""
    quizzes = models.execute_kw(DB, uid, PASSWORD, 'quiz.quiz', 'search_read', 
                              [[]], 
                              {'fields': ['name', 'slug', 'access_mode', 'published']})
    
    return pd.DataFrame(quizzes)

quizzes_df = get_quizzes()
display(quizzes_df)

## List Portal Users

Now let's retrieve all portal users that could potentially access quizzes.

In [ ]:
def get_portal_users():
    """Get all portal users"""
    portal_group_id = models.execute_kw(DB, uid, PASSWORD, 'ir.model.data', 'xmlid_to_res_id', 
                                      ['base.group_portal'])
    
    portal_users = models.execute_kw(DB, uid, PASSWORD, 'res.users', 'search_read', 
                                   [['groups_id', 'in', [portal_group_id]]], 
                                   {'fields': ['id', 'name', 'login', 'email']})
    
    return pd.DataFrame(portal_users)

portal_users_df = get_portal_users()
display(portal_users_df)

## Quiz Access Control Functions

Let's create functions to simplify managing quiz access.

In [ ]:
def set_quiz_access_mode(quiz_id, access_mode):
    """Set the access mode for a quiz
    
    Args:
        quiz_id (int): ID of the quiz
        access_mode (str): One of 'public', 'portal', 'invitation', or 'internal'
    """
    valid_modes = ['public', 'portal', 'invitation', 'internal']
    if access_mode not in valid_modes:
        print(f"Invalid access mode. Choose from: {valid_modes}")
        return
    
    result = models.execute_kw(DB, uid, PASSWORD, 'quiz.quiz', 'write', 
                             [[quiz_id], {'access_mode': access_mode}])
    
    if result:
        print(f"Updated quiz ID {quiz_id} to {access_mode} access mode")
    else:
        print(f"Failed to update quiz ID {quiz_id}")

## Quiz Access Management Dashboard

Now let's create a simple interactive dashboard to manage quiz access.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create dropdown for quizzes
quiz_options = [(f"{row['name']} ({row['access_mode']})", row['id']) for _, row in quizzes_df.iterrows()]
quiz_dropdown = widgets.Dropdown(
    options=quiz_options,
    description='Quiz:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Create radio buttons for access mode
access_mode = widgets.RadioButtons(
    options=['public', 'portal', 'invitation', 'internal'],
    description='Access Mode:',
    disabled=False
)

# Create update button
update_button = widgets.Button(
    description='Update Access Mode',
    button_style='primary',
    tooltip='Click to update the access mode'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        set_quiz_access_mode(quiz_dropdown.value, access_mode.value)
        # Refresh quizzes list
        display(get_quizzes())

update_button.on_click(on_button_clicked)

display(widgets.VBox([quiz_dropdown, access_mode, update_button, output]))

## Simplified User Access Management

Now let's create a tool to easily grant or revoke quiz access for specific portal users.

In [ ]:
def create_access_invitation(quiz_id, partner_id, name=None):
    """Create an access invitation for a specific partner
    
    Args:
        quiz_id (int): ID of the quiz
        partner_id (int): ID of the partner (res.partner)
        name (str, optional): Name of the invitation
    
    Returns:
        int: ID of the created invitation
    """
    if name is None:
        # Get partner name
        partner = models.execute_kw(DB, uid, PASSWORD, 'res.partner', 'read', 
                                  [[partner_id]], {'fields': ['name']})
        # Get quiz name
        quiz = models.execute_kw(DB, uid, PASSWORD, 'quiz.quiz', 'read', 
                               [[quiz_id]], {'fields': ['name']})
        
        name = f"Access for {partner[0]['name']} to {quiz[0]['name']}"
    
    invitation_id = models.execute_kw(DB, uid, PASSWORD, 'quiz.access.invitation', 'create', [{
        'name': name,
        'partner_id': partner_id,
        'quiz_ids': [(4, quiz_id)]
    }])
    
    if invitation_id:
        print(f"Created invitation ID {invitation_id} for partner ID {partner_id} to access quiz ID {quiz_id}")
    else:
        print(f"Failed to create invitation")
        
    return invitation_id

def send_invitation(invitation_id):
    """Send the invitation to the partner"""
    result = models.execute_kw(DB, uid, PASSWORD, 'quiz.access.invitation', 'action_send_invitation', 
                             [[invitation_id]])
    
    print(f"Invitation sent to partner")
    return result

def get_user_partner_id(user_id):
    """Get partner_id for a user"""
    user = models.execute_kw(DB, uid, PASSWORD, 'res.users', 'read', 
                           [[user_id]], {'fields': ['partner_id']})
    
    if user and user[0]['partner_id']:
        return user[0]['partner_id'][0]
    return None

## Interactive User Access Management Tool

Let's build an interactive tool to manage user access to quizzes.

In [ ]:
# Create multiselect for portal users
user_options = [(row['name'], row['id']) for _, row in portal_users_df.iterrows()]
users_select = widgets.SelectMultiple(
    options=user_options,
    description='Portal Users:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%', height='200px')
)

# Create button to grant access
grant_access_button = widgets.Button(
    description='Grant Access',
    button_style='success',
    tooltip='Click to grant access to selected users'
)

user_output = widgets.Output()

def on_grant_access_clicked(b):
    with user_output:
        clear_output()
        quiz_id = quiz_dropdown.value
        
        for user_id in users_select.value:
            partner_id = get_user_partner_id(user_id)
            if partner_id:
                invitation_id = create_access_invitation(quiz_id, partner_id)
                if invitation_id:
                    send_invitation(invitation_id)
            else:
                print(f"Could not find partner for user ID {user_id}")

grant_access_button.on_click(on_grant_access_clicked)

display(widgets.VBox([widgets.Label("Select users to grant quiz access:"), users_select, grant_access_button, user_output]))

## View Current Access Invitations

Let's create a function to view all current invitations and their status.

In [ ]:
def get_access_invitations(quiz_id=None):
    """Get all access invitations"""
    domain = []
    if quiz_id:
        domain = [('quiz_ids', 'in', [quiz_id])]
        
    invitations = models.execute_kw(DB, uid, PASSWORD, 'quiz.access.invitation', 'search_read', 
                                  [domain], 
                                  {'fields': ['name', 'partner_id', 'state', 'token', 'expiration_date']})
    
    # Process partner_id to get just the name
    for inv in invitations:
        if inv['partner_id']:
            inv['partner_name'] = inv['partner_id'][1]
            inv['partner_id'] = inv['partner_id'][0]
        else:
            inv['partner_name'] = ''
    
    return pd.DataFrame(invitations)

# Button to refresh invitations
refresh_button = widgets.Button(
    description='Refresh Invitations',
    button_style='info',
    tooltip='Click to refresh the list of invitations'
)

invitations_output = widgets.Output()

def on_refresh_clicked(b):
    with invitations_output:
        clear_output()
        quiz_id = quiz_dropdown.value
        invitations_df = get_access_invitations(quiz_id)
        display(invitations_df[['name', 'partner_name', 'state', 'token', 'expiration_date']])

refresh_button.on_click(on_refresh_clicked)

display(widgets.VBox([widgets.HTML("<h3>Access Invitations</h3>"), refresh_button, invitations_output]))

## Revoke Access

Finally, let's create a tool to easily revoke access for specific users.

In [ ]:
def get_invited_partners(quiz_id):
    """Get all partners who have been invited to a quiz"""
    invitations = models.execute_kw(DB, uid, PASSWORD, 'quiz.access.invitation', 'search_read', 
                                  [('quiz_ids', 'in', [quiz_id])], 
                                  {'fields': ['partner_id']})
    
    partner_ids = [inv['partner_id'][0] for inv in invitations if inv['partner_id']]
    
    partners = models.execute_kw(DB, uid, PASSWORD, 'res.partner', 'read', 
                               [partner_ids], {'fields': ['id', 'name', 'email']})
    
    return pd.DataFrame(partners)

def revoke_access(quiz_id, partner_id):
    """Revoke access for a partner to a quiz"""
    # Find invitations for this quiz and partner
    invitation_ids = models.execute_kw(DB, uid, PASSWORD, 'quiz.access.invitation', 'search', 
                                    [['&', ('quiz_ids', 'in', [quiz_id]), ('partner_id', '=', partner_id)]])
    
    if invitation_ids:
        # Mark as expired
        result = models.execute_kw(DB, uid, PASSWORD, 'quiz.access.invitation', 'write', 
                                [invitation_ids, {'state': 'expired'}])
        if result:
            print(f"Revoked access for partner ID {partner_id} to quiz ID {quiz_id}")
        else:
            print(f"Failed to revoke access")
    else:
        print(f"No invitations found for partner ID {partner_id} to quiz ID {quiz_id}")

# Create dropdown for partners with access
partners_dropdown = widgets.Dropdown(
    options=[],
    description='Partner:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Create button to revoke access
revoke_button = widgets.Button(
    description='Revoke Access',
    button_style='danger',
    tooltip='Click to revoke access for selected partner'
)

revoke_output = widgets.Output()

# Button to refresh partners list
refresh_partners_button = widgets.Button(
    description='Refresh Partners',
    button_style='info',
    tooltip='Click to refresh the list of partners with access'
)

def on_refresh_partners_clicked(b):
    with revoke_output:
        clear_output()
        quiz_id = quiz_dropdown.value
        partners_df = get_invited_partners(quiz_id)
        
        if not partners_df.empty:
            partners_dropdown.options = [(row['name'], row['id']) for _, row in partners_df.iterrows()]
            display(partners_df)
        else:
            partners_dropdown.options = []
            print("No partners have access to this quiz")

def on_revoke_clicked(b):
    with revoke_output:
        clear_output()
        quiz_id = quiz_dropdown.value
        partner_id = partners_dropdown.value
        
        if partner_id:
            revoke_access(quiz_id, partner_id)
            # Refresh partners list
            on_refresh_partners_clicked(None)
        else:
            print("No partner selected")

refresh_partners_button.on_click(on_refresh_partners_clicked)
revoke_button.on_click(on_revoke_clicked)

display(widgets.VBox([widgets.HTML("<h3>Revoke Access</h3>"), 
                      refresh_partners_button, 
                      partners_dropdown, 
                      revoke_button, 
                      revoke_output]))

## Simplified Access Mode Management

This function allows you to quickly set a quiz to Portal mode and add multiple users at once.

In [ ]:
def setup_portal_quiz(quiz_id, user_ids=[]):
    """Set up a quiz for portal access and grant access to specified users
    
    Args:
        quiz_id (int): ID of the quiz
        user_ids (list): List of user IDs to grant access
    """
    # Set access mode to portal
    set_quiz_access_mode(quiz_id, 'portal')
    
    # Grant access to specified users
    for user_id in user_ids:
        partner_id = get_user_partner_id(user_id)
        if partner_id:
            invitation_id = create_access_invitation(quiz_id, partner_id)
            if invitation_id:
                send_invitation(invitation_id)
        else:
            print(f"Could not find partner for user ID {user_id}")

# Create a button to set up portal access
setup_portal_button = widgets.Button(
    description='Set Up Portal Access',
    button_style='primary',
    tooltip='Click to set up portal access and grant access to selected users'
)

setup_output = widgets.Output()

def on_setup_portal_clicked(b):
    with setup_output:
        clear_output()
        quiz_id = quiz_dropdown.value
        user_ids = users_select.value
        
        setup_portal_quiz(quiz_id, user_ids)
        print(f"Portal access setup completed for quiz ID {quiz_id}")

setup_portal_button.on_click(on_setup_portal_clicked)

display(widgets.VBox([widgets.HTML("<h3>Quick Portal Setup</h3>"), 
                      widgets.Label("This will set the quiz to Portal mode and grant access to selected users:"), 
                      setup_portal_button, 
                      setup_output]))